# Scienti API

In [1]:
import requests
import getpass
import pandas as pd
from IPython.display import JSON
import time

## Obtain products of "Apropiación Social del Conocimiento" (ASC) from Scienti API

### Read `"NIVEL_MAX"` for products of ASC

In [2]:
asc=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vSjpIpwy89MwRwJdXNy0_Z5zCK6XrieZ2_Cf7i1SQTuk4b_hRrNzuCTBgQe3eTOew/pub?output=xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
asc.columns

Index(['COD_RH', 'COD_PRODUCTO', 'SGL_CATEGORIA', 'NIVEL_0', 'NIVEL_1',
       'NIVEL_2', 'NIVEL_3', 'NIVEL_MAX', 'NÚMERO', 'CATEGORIAS', 'ASC', 'NC',
       'FRH', ' ', ' .1', ' .2', ' .3', ' .4', ' .5', ' .6', ' .7', ' .8',
       ' .9', ' .10', ' .11'],
      dtype='object')

In [4]:
asc[:1]

,COD_RH,COD_PRODUCTO,SGL_CATEGORIA,NIVEL_0,NIVEL_1,NIVEL_2,NIVEL_3,NIVEL_MAX,NÚMERO,CATEGORIAS,...,.2,.3,.4,.5,.6,.7,.8,.9,.10,.11
0,150.0,2.0,ART-00,1: Producción bibliográfica,11: Artículo,111: Publicado en revista especializada,NaN,111: Publicado en revista especializada,72886.0,"['ART-00', 'ART-ART', 'ART-GC', 'EPA-EPA', 'PE...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dasc=dict(asc[asc['ASC']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())

In [6]:
dnc=dict(asc[asc['NC']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())
dfrh=dict(asc[asc['FRH']==1]['NIVEL_MAX'].str.split(': ').str[:2].to_list())

### Get all products ids from Scienti API

In [7]:
k=getpass.getpass('API KEY')

API KEY ········


Get full list of pro

In [8]:
institution='udea'
year=2023
p=requests.get(f"http://apis.colav.co/scienti/info/?apikey={k}&get=ids&institution={institution}&model_year={year}")

In [9]:
p.status_code

200

In [10]:
p=p.json()

In [11]:
pd.DataFrame(p[0].get('entities'))

,name,ids
0,network,"[{'COD_RH': '0000186473', 'COD_RED': 4}, {'COD..."
1,product,"[{'COD_RH': '0000000011', 'COD_PRODUCTO': '1'}..."
2,event,"[{'COD_RH': '0000000016', 'COD_EVENTO': 1}, {'..."
3,patent,"[{'COD_RH': '0000203190', 'COD_PATENTE': 1}, {..."
4,institution_endorsement,"[{'COD_AVAL_INSTITUCION': 1382}, {'COD_AVAL_IN..."
5,project,"[{'COD_RH': '0000000011', 'COD_PROYECTO': '1'}..."


In [12]:
p=[d for d in p[0].get('entities') if d.get('name')=='product']

In [13]:
len(p[0].get('ids'))

413913

### Filter products of ASC 

In [17]:
import json

In [18]:
def to_json(d,f):
    ob=open(f,'w')
    json.dump(d,ob)
    ob.close()
    
def read_json(f):
    ob=open(f,'r')
    d=json.load(f)
    ob.close()    

In [19]:
read_json(ii,'i.json')

TypeError: read_json() takes 1 positional argument but 2 were given

In [36]:
f=open('')

'COD_RH'

In [20]:
ws=[]
wn=[]
wf=[]
ii=[]
i=310000#0
for d in p[0].get('ids')[310000:]:   
    w=requests.get(f"http://apis.colav.co/scienti/product?apikey={k}&model_year=2022&institution=udea&COD_RH={d.get('COD_RH')}&COD_PRODUCTO={d.get('COD_PRODUCTO')}")
    if w.status_code!=200:
        continue
    if not w.json():
        continue        
    wj=w.json()
    x=wj.get('COD_TIPO_PRODUCTO')
    if isinstance(x,str) and x in dasc.keys():
        wj['TIPO_PRODUCTO']=dasc.get(x)
        ws.append(wj)
        time.sleep(0.01)
    if isinstance(x,str) and x in dnc.keys():
        wj['TIPO_PRODUCTO']=dnc.get(x)        
        wn.append(wj)
        time.sleep(0.01)
    if isinstance(x,str) and x in dfrh.keys():
        wj['TIPO_PRODUCTO']=dfrh.get(x)        
        wf.append(wj)
        time.sleep(0.01)
        
    print(f"{str(len(ws)).zfill(6)}:{str(len(wn)).zfill(6)}:{str(len(wf)).zfill(6)}/{str(i).zfill(6)}",end='\r')
    if i%10000==0:
        to_json(ws,'asc2.json')
        to_json(wn,'nc2.json')
        to_json(wf,'frh2.json')
        to_json(ii,'i2.json')
    #if len(wf)>0:
    #    break
    i+=1
    ii=[i]

        
to_json(ws,'asc2.json')
to_json(wn,'nc2.json')
to_json(wf,'frh2.json')        

In [66]:
if w.json(): print(1)

In [22]:
len(ws)

34877

In [16]:
import json

In [23]:
fp=open('asc.json','w')
json.dump(ws,fp)
fp.close()

In [18]:
ls -lh asc.json

-rw-r--r-- 1 restrepo restrepo 263M Aug 22 20:59 asc.json


fp=open('asc.json','r')
ws=json.load(fp)
fp.close()

In [78]:
len(ws)

16881

### Export to table

In [21]:
#https://www.techieclues.com/blogs/how-to-get-variable-name-as-string-in-python
def name(variable):
    for name in globals():
        if id(globals()[name]) == id(variable):
            return name
    for name in locals():
        if id(locals()[name]) == id(variable):
            return name
    return None

def details(w):
    d=w.get('details')[0]
    if len(d.keys())==1:
        intd=d.get(list(d.keys())[0])
        if intd and isinstance(intd,list):
            newintd={}
            for intk in intd[0].keys():
                if intd[0][intk]:
                    newintd[intk]=intd[0][intk]
    return newintd

filterfields={
'general':['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO','TIPO_PRODUCTO', 'TXT_NME_PROD', 
       'TXT_RESUMEN_PROD', 
        'NRO_ANO_PRESENTA','NRO_MES_PRESENTA','DTA_CREACION','DTA_ACTUALIZACION',
        'TXT_WEB_PRODUCTO', 'SGL_CATEGORIA',
        'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR',  
       'NRO_AUTORES_TOTAL', 'NRO_GRUPOS',
       'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO'
       ],
'author':['TXT_TOTAL_NAMES','TXT_NAMES_RH',
'TXT_PRIM_APELL',
'TXT_SEG_APELL',
'TPO_NACIONALIDAD',
'TPO_DOCUMENTO_IDENT',
'TPO_SEXO',
'COD_ORCID',
'AUTOR_ID_SCP',
],
'group':['NRO_ID_GRUPO', 'COD_ID_GRUPO', 'NME_GRUPO','COD_RH_AREA', 'COD_AREA_CONHEC', 
       'COD_PROGRAMA', 'COD_PROGRAMA_SECUND','TXT_PLAN_TRABAJO', 
       'TXT_ESTADO_ARTE', 'TXT_OBJETIVOS', 'TXT_PROD_DESTACADA', 'TXT_RETOS', 'TXT_VISION',
       'area'],
'institution':['COD_INST', 'COD_INST_MACRO', 'NME_INST','SGL_INST', 
             'COD_SECTOR_ECON', 'SGL_PAIS', 'SGL_DEPARTAMENTO'],
'authors':['COD_RH_OTRO','TXT_TOTAL_NAMES_FILTRO'],
'project':['COD_RH', 'COD_PROYECTO', 'TXT_NME_PROYECTO', 'TXT_NME_PROYECTO_FILTRO', 'TXT_OBSERV_PROYECTO', 
         'TXT_RESUMEN_PROYECTO', 'TPO_PROYECTO', 'NRO_ANO_INICIO', 
         'NRO_ANO_FIN' 'NRO_DURACION', 'TPO_ESTADO_PROYECTO'],
'article':['TXT_VOLUMEN_REVISTA',
  'TXT_FASCICULO_REVISTA',
  'TXT_SERIE_REVISTA',
  'TXT_PAGINA_INICIAL',
  'TXT_PAGINA_FINAL'],
'journal':['TXT_ISSN_SEP','TXT_NME_REVISTA','SGL_PAIS','TPO_REVISTA','TPO_CLASIFICACION','TXT_URL'],   
'oriented_thesis': ['COD_PROGRAMA_ACADEMICO',
      'NRO_PAGINAS',
      'TPO_ORIENTACION',
      'NRO_MESES',
      'TPO_ESTADO',
      'TXT_NME_ORIENTADOS',
      'VALORACION_OBT_TESIS',
      'TPO_INSTITUCION',
      'NRO_ANO_FIN',
      'NRO_MES_FIN'],
'institution': [
        'NME_INST',
        'SGL_INST',
        'URL_HOME_PAGE',
        'TXT_CIUDAD_INST',
        'ID_INSTITUCION']    
}

In [75]:
f=open('frh.json','r')
wn=json.load(f)
f.close()

In [76]:
Ws=[]
for w in wn:#ws:#wf:
    W={}
    for k in filterfields['general']:
        W[k]=w.get(k)
    for sub in ['author','authors','group','institution','project','details']:
        if w.get(sub) and isinstance(w.get(sub),list):
            if sub != 'details':
                for kt in filterfields[sub]:
                    W[f'{sub}_{kt}']=w.get(sub)[0].get(kt)
            else:
                details=list(w.get(sub)[0].keys())[0]
                detailed=w.get(sub)[0].get(details)[0]
                if details in filterfields.keys():
                    for kt in filterfields[details]:
                        W[f'{details}_{kt}']=w.get(sub)[0].get(details)[0].get(kt)
                else:
                    W['details']=w.get(sub)[0].get(details)[0]
                try:
                    subdetails = [ x for x in detailed.keys() if isinstance(detailed.get(x),list) ][0]
                except:
                    subdetails = None
                    
                if subdetails and subdetails in filterfields.keys():
                    for kt in filterfields[subdetails]:
                        W[f'{subdetails}_{kt}']=w.get(sub)[0].get(details)[0].get(subdetails)[0].get(kt)                    
#                raise Exception('STOP')
    Ws.append(W)
    print(f'{str(len(Ws)).zfill(6)}',end='\r')
del wn#ws#wf

In [78]:
df=pd.DataFrame(Ws)

In [79]:
#df.to_excel('ASC1.xlsx',index=None)
#df[:20000].to_excel('NC1.xlsx',index=None)
#df[20000:40000].to_excel('NC2.xlsx',index=None)
#df[40000:].to_excel('NC3.xlsx',index=None)
df.to_excel('FRH1.xlsx',index=None)

In [70]:
del Ws

In [71]:
ls -lh NC3.xlsx

-rw-r--r-- 1 restrepo restrepo 8.9M Sep 12 09:45 NC3.xlsx


In [72]:
df[:1]

,COD_RH,COD_PRODUCTO,COD_TIPO_PRODUCTO,TIPO_PRODUCTO,TXT_NME_PROD,TXT_RESUMEN_PROD,NRO_ANO_PRESENTA,NRO_MES_PRESENTA,DTA_CREACION,DTA_ACTUALIZACION,...,project_COD_PROYECTO,project_TXT_NME_PROYECTO,project_TXT_NME_PROYECTO_FILTRO,project_TXT_OBSERV_PROYECTO,project_TXT_RESUMEN_PROYECTO,project_TPO_PROYECTO,project_NRO_ANO_INICIO,project_NRO_ANO_FINNRO_DURACION,project_TPO_ESTADO_PROYECTO,details
0,0000000011,10,113,Revisión (Survey),Glucagon like peptide 1 receptor agonists: a t...,None,2018.0,3,2018-03-05 11:07:51,2018-03-05 11:07:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df.columns

Index(['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO', 'TIPO_PRODUCTO',
       'TXT_NME_PROD', 'TXT_RESUMEN_PROD', 'NRO_ANO_PRESENTA',
       'NRO_MES_PRESENTA', 'DTA_CREACION', 'DTA_ACTUALIZACION',
       'TXT_WEB_PRODUCTO', 'SGL_CATEGORIA', 'TXT_DOI', 'TXT_INST_PART',
       'DTA_EXPOSICION', 'STA_EVENTO', 'TXT_NME_TITULAR', 'NRO_AUTORES_TOTAL',
       'NRO_GRUPOS', 'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO',
       'author_TXT_TOTAL_NAMES', 'author_TXT_NAMES_RH',
       'author_TXT_PRIM_APELL', 'author_TXT_SEG_APELL',
       'author_TPO_NACIONALIDAD', 'author_TPO_DOCUMENTO_IDENT',
       'author_TPO_SEXO', 'author_COD_ORCID', 'author_AUTOR_ID_SCP',
       'article_TXT_VOLUMEN_REVISTA', 'article_TXT_FASCICULO_REVISTA',
       'article_TXT_SERIE_REVISTA', 'article_TXT_PAGINA_INICIAL',
       'article_TXT_PAGINA_FINAL', 'journal_TXT_ISSN_SEP',
       'journal_TXT_NME_REVISTA', 'journal_SGL_PAIS', 'journal_TPO_REVISTA',
       'journal_TPO_CLASIFICACION', 'journal_TXT_URL'

In [74]:
del df

In [38]:
from IPython.display import JSON

In [40]:
JSON(df.dropna(subset=['author_TXT_TOTAL_NAMES','group_NRO_ID_GRUPO','institution_NME_INST',
                 'project_TXT_NME_PROYECTO','TXT_RESUMEN_PROD']).iloc[0].to_dict())

<IPython.core.display.JSON object>

/usr/local/lib/python3.9/dist-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [32]:
import unidecode as ud

In [36]:
df.shape

(34877, 52)

In [41]:
df['TXT_NME_PROD'].str.lower().drop_duplicates().shape

(31262,)

In [35]:

df['TXT_NME_PROD'].str.lower().apply(ud.unidecode).drop_duplicates().shape

(31156,)

## Appendix

In [42]:
df=pd.DataFrame(ws)
df.columns

Index(['COD_RH', 'COD_PRODUCTO', 'TXT_NME_PROD', 'TXT_NME_PROD_FILTRO',
       'TXT_RESUMEN_PROD', 'TXT_OBSERV_PROD', 'SGL_IDIOMA',
       'COD_TIPO_PRODUCTO', 'COD_SUBTIPO_PRODUCTO', 'TXT_COMPLEMENTARIO',
       'COD_TIPO_PRODUCTO_MDCN', 'SGL_PAIS', 'NRO_ANO_PRESENTA',
       'NRO_MES_PRESENTA', 'TXT_WEB_PRODUCTO', 'NRO_IND_EXISTENCIA_MDCN',
       'NRO_IND_CALIDAD_MDCN', 'TXT_IND_CALIDAD_MDCN',
       'NRO_IND_VISIBILIDAD_MDCN', 'TPO_AMPERSAN_MDCN',
       'TPO_MEDIO_DIVULGACION', 'STA_RELEVANCIA', 'COD_PRODUCTO_BNR',
       'STA_CERTIFICADO', 'STA_VERIFICACION_AUTO', 'DTA_VERIF_AUTO',
       'STA_VERIFICACION_USUA', 'DTA_VERIF_USUA', 'TXT_FICHA_RESUMEN',
       'DTA_CREACION', 'DTA_ACTUALIZACION', 'STA_VENTANA', 'NRO_PATENTES',
       'NRO_SECRETOS', 'COD_UNO_MDCN', 'COD_DOS_MDCN', 'COD_TRES_MDCN',
       'STA_VENTANA_2', 'TXT_IND_CALIDAD_MDCN_2', 'TIPO_PRODUCTO_MDCN',
       'TIPO_PRODUCTO_MDCN_2', 'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR'

In [ ]:
general=['COD_RH', 'COD_PRODUCTO', 'COD_TIPO_PRODUCTO','TIPO_PRODUCTO', 'TXT_NME_PROD', 
       'TXT_RESUMEN_PROD', 
        'NRO_ANO_PRESENTA',
        'TXT_WEB_PRODUCTO', 
       'TXT_FICHA_RESUMEN',
        'TXT_DOI', 'TXT_INST_PART', 'DTA_EXPOSICION',
       'STA_EVENTO', 'TXT_NME_TITULAR',  'author', 'product_type',
       'author_others', 'details', 'language',  '
       'NRO_AUTORES_TOTAL', 'NRO_GRUPOS',  'group', 'institution', 'project', 
       'COD_PROYECTO', 'ANO_INIC', 'ANO_FIM', 'TXT_NOME_PROJETO',
       ]

In [49]:
df['author'].iloc[0][0].keys()

dict_keys(['COD_RH', 'NRO_ID_CNPQ', 'TXT_NAMES_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES', 'TXT_NAMES_RH_FILTRO', 'TXT_PRIM_APELL_FILTRO', 'TXT_SEG_APELL_FILTRO', 'TXT_TOTAL_NAMES_FILTRO', 'TPO_NACIONALIDAD', 'DTA_NACIM', 'TPO_DOCUMENTO_IDENT', 'TPO_SEXO', 'COD_RH_MUN_NACIM', 'COD_MUN_NACIM', 'NRO_DOCUMENTO_IDENT', 'COD_RH_MUN_EXPED_DOC', 'COD_MUN_EXPED_DOC', 'NRO_LIBRETA_MILITAR', 'TPO_LIBRETA_MILITAR', 'NRO_PASAPORTE', 'NRO_VISA', 'TPO_VISA', 'DTA_INI_VIGENCIA_VISA', 'DTA_FIN_VIGENCIA_VISA', 'COD_RH_RELIGION', 'COD_RELIGION', 'SGL_PAIS_NACIM', 'TXT_AFICION', 'TXT_CONGREGA_PERTENECE', 'TXT_CONGREGA_PERTENECIO', 'NRO_CEDULA_EXTRANJ', 'TXT_COMENT_SUGEREN', 'DTA_DEFUNC', 'COD_NIVEL_FORMACION', 'TPO_ESTADO_CIVIL', 'TPO_DIRECCION_USUAL', 'TPO_DOC_ACTUALIZACION', 'TPO_VER_DOC_ACTUALIZACION', 'TXT_CITACION_BIBLIO', 'TXT_CITACION_BIBLIO_FILTRO', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'TXT_RESPUESTA', 'TXT_PREGUNTA', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'STA_

In [ ]:
author=['author_TXT_TOTAL_NAMES']

In [53]:
df['group'].dropna().iloc[0][0].keys()

dict_keys(['NRO_ID_GRUPO', 'COD_ID_GRUPO', 'NME_GRUPO', 'NME_GRUPO_FILTRO', 'ANO_FORMACAO', 'MES_FORMACAO', 'COD_RH_AREA', 'COD_AREA_CONHEC', 'COD_PROGRAMA', 'COD_PROGRAMA_SECUND', 'TPO_ESTADO_GR', 'DTA_ESTADO_GR', 'TXT_CLASIF', 'DTA_CLASIF', 'DTA_FIN_CLASIF', 'TPO_GRUPO', 'DTA_TPO_GRUPO', 'STA_ELIMINADO', 'TXT_PLAN_TRABAJO', 'TXT_ESTADO_ARTE', 'TXT_OBJETIVOS', 'TXT_PROD_DESTACADA', 'TXT_RETOS', 'TXT_VISION', 'TXT_SUGERENCIAS', 'XML_FICHA_RSM', 'STA_CERTIFICADO', 'DTA_CERTIFICADO', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_CERTIFICACION', 'COD_SESION', 'DTA_INI_SESION', 'TXT_CLASIF_1', 'COD_GRAN_AREA_CONHEC_MED', 'STA_CENTRO', 'TPO_CENTRO', 'STA_SECTOR', 'NRO_INTENTOS', 'STA_ACTIVO', 'COD_ACTIVACION', 'NRO_BLOQUEOS', 'COD_AREA_CONHEC_BK', 'institution', 'knowledge_area'])

In [ ]:
group=['group_NRO_ID_GRUPO', 'group_COD_ID_GRUPO', 'group_NME_GRUPO','group_COD_RH_AREA', 'group_COD_AREA_CONHEC', 
       'group_COD_PROGRAMA', 'group_COD_PROGRAMA_SECUND','group_TXT_PLAN_TRABAJO', 
       'group_TXT_ESTADO_ARTE', 'group_TXT_OBJETIVOS', 'group_TXT_PROD_DESTACADA', 'group_TXT_RETOS', 'group_TXT_VISION', 'group_TXT_SUGERENCIAS']


In [58]:
df['institution'].dropna().iloc[0][0].keys()

dict_keys(['COD_INST', 'COD_INST_MACRO', 'NME_INST', 'NME_INST_FILTRO', 'SGL_INST', 'COD_SECTOR_ECON', 'SGL_PAIS', 'SGL_DEPARTAMENTO', 'COD_RH_MUNICIPIO', 'COD_MUNICIPIO', 'URL_HOME_PAGE', 'TXT_DIRECCION', 'TXT_TELEFONO', 'TXT_FAX', 'TXT_EMAIL', 'TXT_NIT', 'TXT_DIGITO_VERIFICADOR', 'ID_REPRESENTANTE', 'TPO_AVAL', 'TXT_CIUDAD_INST', 'TPO_CARACTER', 'TPO_CONTRIBUYENTE', 'TXT_APARTADO_AEREO', 'STA_EXPORTA', 'COD_NIVEL', 'COD_IES', 'DTA_CONSTITUCION', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'COD_SECTOR_ECON2', 'ID_INSTITUCION', 'STA_TERMINOS_COND', 'DTA_ACEPTA_TERMINOS'])

In [ ]:
institution=['institution_COD_INST', 'institution_COD_INST_MACRO', 'institution_NME_INST','institution_SGL_INST', 'institution_COD_SECTOR_ECON', 'institution_SGL_PAIS', 'institution_SGL_DEPARTAMENTO']

In [59]:
df['author_others'].dropna().iloc[0][0].keys()

dict_keys(['COD_RH_CREA', 'COD_RH_OTRO', 'TXT_NME_RH', 'TXT_PRIM_APELL', 'TXT_SEG_APELL', 'TXT_TOTAL_NAMES_FILTRO', 'SGL_PAIS', 'TPO_NACIONALIDAD', 'DTA_NACIMIENTO', 'TPO_DOC_IDENTIFICACION', 'NRO_DOC_IDENTIFICACION', 'NRO_ID_CNPQ', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'NRO_DOC_FILTRO', 'TXT_TOTAL_NAMES', 'TXT_NME_RH_FILTRO', 'TXT_PRIM_APELL_FILTRO', 'TXT_SEG_APELL_FILTRO', 'COD_RH_REF', 'COD_ORCID', 'AUTOR_ID_SCP'])

In [ ]:
authors=['authors_COD_RH_OTRO','authors_TXT_TOTAL_NAMES_FILTRO']

In [72]:
df['project'].dropna().iloc[0][0].keys()

dict_keys(['COD_RH', 'COD_PROYECTO', 'TXT_NME_PROYECTO', 'TXT_NME_PROYECTO_FILTRO', 'TXT_OBSERV_PROYECTO', 'TXT_RESUMEN_PROYECTO', 'TPO_PROYECTO', 'NRO_MES_INICIO', 'NRO_ANO_INICIO', 'NRO_MES_FIN', 'NRO_ANO_FIN', 'NRO_MES_PROYEC', 'NRO_ANO_PROYEC', 'NRO_DURACION', 'TPO_ESTADO_PROYECTO', 'NRO_GRADUACION', 'NRO_ESPECIALIZACION', 'NRO_MAESTRIA_ACAD', 'NRO_MAESTRIA_PROF', 'NRO_DOCTORADO', 'COD_PROYECTO_BNR', 'STA_CERTIFICADO', 'STA_VERIFICACION_AUTO', 'DTA_VERIF_AUTO', 'STA_VERIFICACION_USUA', 'DTA_VERIF_USUA', 'TXT_FICHA_RESUMEN', 'DTA_CREACION', 'DTA_ACTUALIZACION', 'TXT_URL', 'COD_PROYECTO_SNCTI', 'TPO_FINANCIACION', 'TPO_FTE_FINAN', 'TPO_AMB_FINAN', 'STA_AVAL_INST', 'DTA_AVAL_INST', 'TPO_AVAL_INST', 'COD_INST_AVALA', 'ID_USUARIO_AVALA', 'DTA_ACTUALIZACION_CVLAC', 'CAT_MED', 'SGL_CATEGORIA'])

In [ ]:
project=['project_COD_RH', 'project_COD_PROYECTO', 'project_TXT_NME_PROYECTO', 'project_TXT_NME_PROYECTO_FILTRO', 'project_TXT_OBSERV_PROYECTO', 
         'project_TXT_RESUMEN_PROYECTO', 'project_TPO_PROYECTO', 'project_NRO_ANO_INICIO', 
         'project_NRO_ANO_FIN' 'project_NRO_DURACION', 'project_TPO_ESTADO_PROYECTO']

In [67]:
df['details'].dropna()

0        [{'technical': [{'COD_RH': '0000000011', 'COD_...
1        [{'course': [{'COD_RH': '0000000011', 'COD_PRO...
2        [{'course': [{'COD_RH': '0000000011', 'COD_PRO...
3        [{'technical': [{'COD_RH': '0000000011', 'COD_...
4        [{'course': [{'COD_RH': '0000000011', 'COD_PRO...
                               ...                        
16876    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16877    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16878    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16879    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
16880    [{'course': [{'COD_RH': '0000771899', 'COD_PRO...
Name: details, Length: 16426, dtype: object